K-Nearest Neighbours Algorithm

In [1]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import math

import operator

In [2]:
from sklearn.preprocessing import LabelEncoder

# Read dataset
df = pd.read_csv('../data/BCCC-CIRA-CIC-DoHBrw-2020.csv')


# Check for null values
#print(df.isnull().sum())

# Fill null values with mean
if df.isnull().sum().sum() > 0:
    df = df.fillna(df.mean())

# Encode categorical data
encoder = LabelEncoder()
df['Label'] = encoder.fit_transform(df['Label'])


sample_size = int(len(df) / 45)  # Specify the desired sample size
print("Sample size: " + str(sample_size))
df_full = df
df = df_full.sample(n=sample_size, random_state=13)  # Randomly sample 'sample_size' number of rows

print(encoder.classes_)

df.describe()
#df.head()  #Displaying first five rows of the dataset

Sample size: 11091
['Benign' 'Malicious']


,FlowBytesSent,FlowSentRate,FlowBytesReceived,FlowReceivedRate,PacketLengthVariance,PacketLengthStandardDeviation,PacketLengthMean,PacketLengthMedian,PacketLengthMode,PacketLengthSkewFromMedian,...,PacketTimeCoefficientofVariation,ResponseTimeTimeVariance,ResponseTimeTimeStandardDeviation,ResponseTimeTimeMean,ResponseTimeTimeMedian,ResponseTimeTimeMode,ResponseTimeTimeSkewFromMedian,ResponseTimeTimeSkewFromMode,ResponseTimeTimeCoefficientofVariation,Label
count,1.109100e+04,1.109100e+04,1.109100e+04,1.109100e+04,1.109100e+04,11091.000000,11091.000000,11091.000000,11091.000000,11091.000000,...,11091.000000,11091.000000,11091.000000,11091.000000,11091.000000,11091.000000,11091.000000,11091.000000,11091.000000,11091.000000
mean,3.977599e+04,4.839952e+04,4.205765e+04,3.245092e+04,9.316694e+04,219.788588,172.843818,94.825603,70.695970,0.449191,...,0.975363,1.648877,0.313990,0.457290,0.406550,0.227879,-1.012756,-0.081031,1.104810,0.492381
std,1.586920e+05,4.246503e+05,1.394485e+05,2.605488e+05,1.564252e+05,211.806057,86.728905,32.088784,14.541817,1.549672,...,0.520421,10.366514,1.217782,2.134924,2.389304,1.822809,3.199509,3.248361,1.862287,0.499964
min,5.500000e+01,1.488526e+00,5.400000e+01,1.576258e+00,0.000000e+00,0.000000,56.000000,54.000000,54.000000,-10.000000,...,0.101544,0.000000,0.000000,0.000007,0.000003,0.000000,-10.000000,-10.000000,0.000000,0.000000
25%,6.180000e+02,5.417957e+01,4.580000e+02,1.417620e+02,4.692099e+02,21.661253,92.000000,76.000000,66.000000,0.204922,...,0.578226,0.000016,0.003774,0.010604,0.012392,0.000016,-1.805785,0.409517,0.529790,0.000000
50%,1.807000e+03,3.677526e+02,4.827000e+03,4.314236e+02,1.798912e+04,134.123537,151.950908,87.000000,68.000000,0.986397,...,0.757208,0.000076,0.008696,0.015145,0.015408,0.000034,-0.027153,0.926987,0.797760,0.000000
75%,5.292000e+03,3.651169e+03,7.729000e+03,4.213958e+03,1.421989e+05,377.092628,228.758621,105.000000,68.000000,1.201609,...,1.515429,0.000326,0.017899,0.024971,0.016556,0.015098,0.905956,1.307290,1.187231,1.000000
max,6.209568e+06,6.176471e+06,4.293889e+06,3.882353e+06,1.369730e+06,1170.354496,586.833333,287.000000,213.000000,2.879882,...,4.171592,144.281231,11.972824,28.017596,28.017596,28.017596,2.770685,3.558791,58.760508,1.000000


Train-Test Split

In [3]:
np.random.seed(1)
msk = np.random.rand(
    len(df)) < 0.5  #An array containing True(with probability 0.5) and False
train = df[msk]  #Rows having array value true
test = df[~msk]  #Rows having array value False
print('Number of observations in the training data: ', len(train))
print('Number of observations in the test data: ', len(test))

Number of observations in the training data:  5514
Number of observations in the test data:  5577


In [4]:
train.head()

,FlowBytesSent,FlowSentRate,FlowBytesReceived,FlowReceivedRate,PacketLengthVariance,PacketLengthStandardDeviation,PacketLengthMean,PacketLengthMedian,PacketLengthMode,PacketLengthSkewFromMedian,...,PacketTimeCoefficientofVariation,ResponseTimeTimeVariance,ResponseTimeTimeStandardDeviation,ResponseTimeTimeMean,ResponseTimeTimeMedian,ResponseTimeTimeMode,ResponseTimeTimeSkewFromMedian,ResponseTimeTimeSkewFromMode,ResponseTimeTimeCoefficientofVariation,Label
44565,171,2579.556192,171,2579.556192,380.250000,19.500000,85.500000,85.5,66,0.000000,...,0.814335,0.000000e+00,0.000000,0.025227,0.025227,0.025227,-10.000000,-10.000000,0.000000,0
82229,1085,2732.605142,4099,10323.454819,215000.869806,463.681863,272.842105,68.0,68,1.325319,...,0.807566,9.919523e-05,0.009960,0.009760,0.007710,0.000003,0.617590,0.979633,1.020476,1
63378,110,2.441408,132,2.929690,30.250000,5.500000,60.500000,60.5,55,0.000000,...,0.999444,1.066619e-09,0.000032,0.025048,0.025048,0.025015,0.000000,1.000000,0.001295,0
238800,293125,2496.034998,169118,1440.083400,14389.935956,119.958059,166.095221,68.0,68,2.453238,...,0.582091,3.346842e-10,0.000018,0.000036,0.000038,0.000038,-0.361898,-0.120633,0.511115,1
395083,353,82.875157,327,76.771038,487.750000,22.085063,85.000000,78.0,66,0.950869,...,0.577349,1.446222e-09,0.000038,0.000053,0.000032,0.000020,1.630325,0.858988,0.718952,0


In [5]:
test.head()

,FlowBytesSent,FlowSentRate,FlowBytesReceived,FlowReceivedRate,PacketLengthVariance,PacketLengthStandardDeviation,PacketLengthMean,PacketLengthMedian,PacketLengthMode,PacketLengthSkewFromMedian,...,PacketTimeCoefficientofVariation,ResponseTimeTimeVariance,ResponseTimeTimeStandardDeviation,ResponseTimeTimeMean,ResponseTimeTimeMedian,ResponseTimeTimeMode,ResponseTimeTimeSkewFromMedian,ResponseTimeTimeSkewFromMode,ResponseTimeTimeCoefficientofVariation,Label
419667,21253,1791.528294,20084,1680.815713,21633.071300,147.080476,154.081819,114.0,115,0.817630,...,0.642373,0.001120,0.030383,0.020175,0.014658,0.000048,0.589664,0.792956,1.495806,0
188161,1713,56.879651,4513,149.852811,157687.878400,397.099331,249.040000,85.0,54,1.239287,...,1.984219,0.000033,0.005743,0.012714,0.015640,0.000324,-1.521796,2.162560,0.452972,0
430708,4240,16431.472789,5281,20466.396338,14223.433452,119.255982,147.189859,105.0,66,1.060667,...,0.470846,0.000081,0.009007,0.010012,0.010543,0.000008,-0.177540,1.110599,0.900182,0
185640,171,10953.814618,105,6726.026520,338.000000,18.384776,92.000000,105.0,105,-2.121320,...,0.707107,0.000000,0.000000,0.015591,0.015591,0.015591,-10.000000,-10.000000,0.000000,0
303468,28669,238.383453,46671,388.070533,22288.089011,149.291959,130.120898,87.0,56,0.866508,...,0.548779,0.000081,0.008990,0.008740,0.010972,0.000009,-0.744985,0.971201,1.028592,1


Euclidean distance

In [6]:
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)

Calculating the Nearest Neighbors

In [7]:
def getNeighbors(trainingSet, testInstance, k):
    distances = []  #List to store all the distance values
    length = len(testInstance) - 1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)  #Calculating the Euclidean Distance
        distances.append((trainingSet[x],dist))  #Appending the distance values to the 'distance' list
    distances.sort(key=operator.itemgetter(1))  #Sorting based on the disance value
    neighbors = []  #List to store all the neighbors
    for x in range(k):
        neighbors.append(distances[x][0])  #Number of neighbors is dependent on the value of k
    return neighbors

In [8]:
def getResponse(neighbors):
    classVotes = {}  #Dictionary to store labels with their counts
    for x in range(len(neighbors)):
        response = neighbors[x][-1]  #Label value of the neighbors
        if response in classVotes:
            classVotes[response] += 1  #if label value is already present increment it by 1
        else:
            classVotes[response] = 1  #If the label value is not yet present add it to the dictionary
    sortedVotes = sorted(
        classVotes.items(), key=operator.itemgetter(1), reverse=True
    )  #Sort the dictinary based on the count value in descending order
    return sortedVotes[0][0]  #Return the label with highest number of occurences

Calculating the accuracy

In [9]:
def getAccuracy(testSet, predictions):
    correct = 0  #Variable to store the correct predictions
    for x in range(len(testSet)):
        if testSet[x][-1] is predictions[x]:  #Checking whether the predicted value is same as label value
            correct += 1  #Incremented when both values are same
    return (correct / float(len(testSet))) * 100.0  #Accuracy = No. of Correct pred / Total number of pred

Implementation

In [10]:
predictions = []  #List to store the predicted values
k = 3  # 3-Nearest Neighbors
trainingSet = train.values.tolist()  #List containing training data
testSet = test.values.tolist()  #List containing test data
for x in range(len(testSet)):
    neighbors = getNeighbors(trainingSet, testSet[x], k)
    result = getResponse(neighbors)
    predictions.append(result)  # Storing the predicted values
    # print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))

Confusion Matrix

In [ ]:
print("Confusion Matrix")
y_test = []
for i in testSet:
    y_test.append(i[-1])
from sklearn.metrics import confusion_matrix, accuracy_score
res = confusion_matrix(y_test, predictions)
print(res)

Confusion Matrix
[[2759  137]
 [  96 2585]]


Metrics for performance evaluation

In [ ]:
accuracy = accuracy_score(y_test, predictions)
print('Accuracy: ' + repr(accuracy) + '%')

TP = res[0, 0]
FP = res[0, 1]
FN = res[1, 0]
TN = res[1, 1]

precision = TP / (TP + FP)
print('Precision: ' + repr(precision))

recall = TP / (TP + FN)
print('Recall: ' + repr(recall))

f1_score = 2 * TP / (2 * TP + FP + FN)
print('F1-score: ' + repr(f1_score))

Accuracy: 0.9582212659135736%
Precision: 0.9526933701657458
Recall: 0.9663747810858143
F1-score: 0.9594853069031473
